# import

In [1]:
# !/opt/apps/apps/binapps/anaconda3/2019.07-numpy-fix/bin/python -m pip install --user  --upgrade pip

In [2]:
# !/opt/apps/apps/binapps/anaconda3/2019.07-numpy-fix/bin/python -m pip install --user tensorflow==1.15


In [3]:
# !/opt/apps/apps/binapps/anaconda3/2019.07-numpy-fix/bin/python -m pip install --user opencv-python

In [1]:
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cv2 as cv
import cv2
# from google.colab.patches import cv2_imshow
import numpy as np

import tensorflow.compat.v1 as tf
import os, argparse
import cv2
import pandas as pd
# from tqdm import tqdm
import json

## Load other dataset

In [5]:
# covid-chestxray-dataset
# !wget https://github.com/ieee8023/covid-chestxray-dataset/archive/refs/heads/master.zip

In [6]:
# !unzip master.zip

# Method

## prepare the model

In [2]:
def crop_top(img, percent=0.15):
    offset = int(img.shape[0] * percent)
    return img[offset:]


def central_crop(img):
    size = min(img.shape[0], img.shape[1])
    offset_h = int((img.shape[0] - size) / 2)
    offset_w = int((img.shape[1] - size) / 2)
    return img[offset_h:offset_h + size, offset_w:offset_w + size]


def process_image_file(filepath, top_percent, size):
    if filepath.split(".")[-1] == "npy":
        x_arr = np.load(filepath)
        x_arr = x_arr * 255.0 / x_arr.max()
        img = np.stack((x_arr, x_arr, x_arr), axis=-1)
    else:
        img = cv2.imread(filepath)
    
    if img is None:
      img = cv2.imread("white.png")
      

    processed_img = crop_top(img, percent=top_percent)
    processed_img = central_crop(processed_img)
    processed_img = cv2.resize(processed_img, (size, size))
    return processed_img, img


'''/////////////////////////main_grad/////////////////////////////'''

def main_grad(args_impath):
  # # add more photo
  # args_impath = "./covid-chestxray-dataset-master/images/000001-1.jpg"
  image_tensor = graph.get_tensor_by_name(args_in_tensorname)
  pred_tensor = sess.graph.get_tensor_by_name('dense_3/Softmax:0') # args_out_tensorname 'norm_dense_2/Softmax:0'
  gradCam = GradCAM(graph=graph, classes = [0,1,2], outLayer=args_out_tensorname, targetLayer=final_conv_name)

  grads = gradCam.compute_grads()
  
  x, origin_im = process_image_file(args_impath, args_top_percent, args_input_size)
  img_arr = np.asanyarray(x)
  size_upsample = (origin_im.shape[1],origin_im.shape[0]) # (w, h)

  x = x.astype('float32') / 255.0
  output, grads_val = sess.run([gradCam.target, grads[mapping[args_pred_class]]], feed_dict={image_tensor: np.expand_dims(x, axis=0)})


  # x = process_image_file(IMAGE_PATH, args_input_size, top_percent=args_top_percent)
  # x = x.astype('float32') / 255.0
  feed_dict = {image_tensor: np.expand_dims(x, axis=0)}
  pred = sess.run(pred_tensor, feed_dict=feed_dict)
  print("pred", pred)

  print('Prediction: {}'.format(inv_mapping[pred.argmax(axis=1)[0]]))
  print('Confidence')
  print(' '.join('{}: {:.3f}'.format(cls.capitalize(), pred[0][i]) for cls, i in mapping.items()))
#   print('**DISCLAIMER**')
#   print('Do not use this prediction for self-diagnosis. You should check with your local authorities for the latest advice on seeking medical assistance.')


  cam3 = generate_cam(output[0],grads_val[0],size_upsample)

  # Overlay cam on image
  cam3 = np.uint8(255*cam3)
  cam3 = cv2.applyColorMap(cam3, cv2.COLORMAP_JET)
  
  im_name = args_impath.split("/")[-1]
  ext = im_name.split(".")[-1]


  # show!!!!
  cam3_resize = cv.resize(cam3, (224, 224))
  
  # cv2.imwrite("cam-1.png", cam3_resize)
  

  new_im = cam3*0.3 + origin_im*0.5


  # Save the GradCAM
  # cv2.imwrite("./output/"+os.path.join(args_outdir, "{}_{}png".format(args_pred_class, im_name.rstrip(ext))), new_im)

  # print("GradCAM image is save in ", args_outdir)

  # show images:
  # imagepath = "assets/ex-covid.jpeg"
  img_m = cv.imread(args_impath)
  if img_m is None:
    img_m = cv.imread("white.png")
    img_m = cv.resize(img_m, (224, 224))
  else:
    img_m = cv.resize(img_m, (224, 224))

  # display(Image(imagepath))
  # cv2_imshow(img_m)  # oringinal
  new_im_2 = cv.resize(new_im, (224, 224))
  # cv2_imshow(new_im_2)  # grad-cam

  # cv2_imshow(cam3_resize) # grad-cam-explain

  return img_m, new_im_2, cam3_resize, pred

'''/////////////////////////main_grad/////////////////////////////'''

'''/////////////////////////bb_iou/////////////////////////////'''

def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, (xB - xA + 1)) * max(0, (yB - yA + 1))
	# print("interarea:", interArea)
	# print("xA, yA, xB, yB:", xA, yA, xB, yB)
  
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = ((boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1))
	boxBArea = ((boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1))
	# print("boxAArea:", boxAArea)
	# print("boxBArea", boxBArea)

	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# print("boxAArea + boxBArea - interArea: ", boxAArea + boxBArea - interArea)
	# print("iou: ", iou)
  
	# return the intersection over union value
	return iou

'''/////////////////////////bb_iou/////////////////////////////'''






class GradCAM:
    def __init__(self, graph, classes, outLayer, targetLayer=None):
        self.graph = graph
        self.classes = classes
        self.targetLayer = targetLayer
        self.outLayer = outLayer

        if self.targetLayer is None:
            self.target = self.find_target_tensor()
        else:
            self.target = self.graph.get_tensor_by_name(self.targetLayer)

    def find_target_tensor(self):
        """
        Find the last tensor that have 4D shape if targetLayer is not specified.
        :return:
        """
        tensor_names = [t.name for op in tf.get_default_graph().get_operations() for t in op.values() if
                   "save" not in str(t.name)]
        for tensor_name in reversed(tensor_names):
            tensor = self.graph.get_tensor_by_name(tensor_name)
            if len(tensor.shape) == 4:
                return tensor

        raise ValueError("Could not find 4D layer. Cannot apply GradCAM")

    def compute_grads(self):
        results = {} # grads of classes with keys being classes and values being normalized gradients
        for classIdx in self.classes:
            one_hot = tf.sparse_to_dense(classIdx, [len(self.classes)], 1.0)
            signal = tf.multiply(self.graph.get_tensor_by_name(self.outLayer),one_hot)
            loss = tf.reduce_mean(signal)

            grads = tf.gradients(loss, self.target)[0]

            norm_grads = tf.div(grads, tf.sqrt(tf.reduce_mean(tf.square(grads)))+tf.constant(1e-5))

            results[classIdx] = norm_grads

        return results


def generate_cam(conv_layer_out, grads_val, upsample_size):
    weights = np.mean(grads_val, axis=(0,1))
    cam = np.zeros(conv_layer_out.shape[0:2], dtype=np.float32)

    # Weight averaginng
    for i, w in enumerate(weights):
        cam += w*conv_layer_out[:,:,i]

    # Apply reLU
    cam = np.maximum(cam, 0)
    cam = cam/np.max(cam)
    cam = cv2.resize(cam, upsample_size)

    # Convert to 3D
    cam3 = np.expand_dims(cam, axis=2)
    cam3 = np.tile(cam3,[1,1,3])

    return cam3





## Prepare 10/30 images

In [3]:
# img_num_set = [29, 51, 73, 75, 100, 125, 179, 202, 251, 257]
# '''
# img_num_set = [29, 51, 73, 75, 100, 125, 179, 202, 251, 257,
#                 288, 297, 325, 361, 369, 480, 498, 509, 524, 642,
#                659, 720, 757, 763, 823, 845, 881, 890, 919, 942]
img_num_set = [51, 179,
                297, 361, 480, 
               659, 720, 823, 919, 942]
# '''
# 53: radiopaedia_org_covid-19-pneumonia-7_85703_0-dcm.nii.gz unknown
# 243: radiopaedia_org_covid-19-pneumonia-10_85902_3-dcm.nii.gz unknown
# 178: no pattern recognized
# 478: 8da14fff.jpg(line480): no pattern recognized
# 479: 1052b0fe.jpg: no pattern recognized
# 514: 000002-11-a.jpg(line516)
# 516: 000001-3.png
# 836: 0ac7580d.jpg
# 877: 006f3a8a.jpg
# 913: 3e9d9c9b02b9bcd81c851134667e76_jumbo.jpg
# 921: 49eca411aa2d4ccd73df21a209a551_jumbo.jpeg

In [4]:
# import metedata

metadata_covid_chest = './covid-chestxray-dataset-master/metadata.csv'
pd_reader = pd.read_csv(metadata_covid_chest)
print(pd_reader)

    patientid  offset  sex   age                         finding  \
0           2     0.0    M  65.0        Pneumonia/Viral/COVID-19   
1           2     3.0    M  65.0        Pneumonia/Viral/COVID-19   
2           2     5.0    M  65.0        Pneumonia/Viral/COVID-19   
3           2     6.0    M  65.0        Pneumonia/Viral/COVID-19   
4           4     0.0    F  52.0        Pneumonia/Viral/COVID-19   
5           4     5.0    F  52.0        Pneumonia/Viral/COVID-19   
6           5     NaN  NaN   NaN                       Pneumonia   
7           6     0.0  NaN   NaN        Pneumonia/Viral/COVID-19   
8           6     4.0  NaN   NaN        Pneumonia/Viral/COVID-19   
9           3     4.0    M  74.0            Pneumonia/Viral/SARS   
10          3     9.0    M  74.0            Pneumonia/Viral/SARS   
11          3    10.0    M  74.0            Pneumonia/Viral/SARS   
12          7     7.0    F  29.0            Pneumonia/Viral/SARS   
13          7    12.0    F  29.0            Pneu

In [5]:
image_name_all =pd_reader['filename']
PCR_result_all = pd_reader['RT_PCR_positive']

In [6]:
pd_reader.index.rename('index', inplace=True)
# lines = pd_reader2['index']
pd_reader

,patientid,offset,sex,age,finding,RT_PCR_positive,survival,intubated,intubation_present,went_icu,...,date,location,folder,filename,doi,url,license,clinical_notes,other_notes,Unnamed: 29
index,,,,,,,,,,,,,,,,,,,,,
0,2,0.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,...,"January 22, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
1,2,3.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,...,"January 25, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
2,2,5.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,...,"January 27, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
3,2,6.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,...,"January 28, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
4,4,0.0,F,52.0,Pneumonia/Viral/COVID-19,Y,NaN,N,N,N,...,"January 25, 2020","Changhua Christian Hospital, Changhua City, Ta...",images,nejmc2001573_f1a.jpeg,10.1056/NEJMc2001573,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,diffuse infiltrates in the bilateral lower lungs,NaN,NaN
5,4,5.0,F,52.0,Pneumonia/Viral/COVID-19,Y,NaN,N,N,N,...,"January 30, 2020","Changhua Christian Hospital, Changhua City, Ta...",images,nejmc2001573_f1b.jpeg,10.1056/NEJMc2001573,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,progressive diffuse interstitial opacities and...,NaN,NaN
6,5,NaN,NaN,NaN,Pneumonia,NaN,NaN,Y,Y,Y,...,2017,NaN,images,ARDSSevere.png,NaN,https://en.wikipedia.org/wiki/File:ARDSSevere.png,CC BY-SA,Severe ARDS. Person is intubated with an OG in...,NaN,NaN
7,6,0.0,NaN,NaN,Pneumonia/Viral/COVID-19,Y,NaN,Y,Y,Y,...,"January 6, 2020","Wuhan Jinyintan Hospital, Wuhan, Hubei Provinc...",images,lancet-case2a.jpg,10.1016/S0140-6736(20)30211-7,https://www.thelancet.com/journals/lancet/arti...,NaN,Case 2: chest x-ray obtained on Jan 6 (2A). Th...,NaN,NaN
8,6,4.0,NaN,NaN,Pneumonia/Viral/COVID-19,Y,NaN,Y,Y,Y,...,"January 10, 2020","Wuhan Jinyintan Hospital, Wuhan, Hubei Provinc...",images,lancet-case2b.jpg,10.1016/S0140-6736(20)30211-7,https://www.thelancet.com/journals/lancet/arti...,NaN,Case 2: chest x-ray obtained on Jan 6 (2A). Th...,NaN,NaN


In [7]:
# # example 10
# for i in range(10):
#   num = img_num_set[i]
#   print("Num#:", num)
#   print(image_name_all[num])
#   temp_name = pd_reader['filename'][num]
#   print("Current img:", temp_name)
#   main_grad('./covid-chestxray-dataset-master/images/' + temp_name)
#   # main_grad('./covid-chestxray-dataset-master/images/' + image_name_all[num])
#   print(i, PCR_result_all[num])

## Prepare graph model

In [8]:
tf.disable_v2_behavior()
weightspath = 'models/COVIDNet-CXR-Large/' 
  
# arguments:
IMAGE_PATH ="assets/ex-covid.jpeg"
output_node_names = "dense_3/Softmax:0" #norm_dense_2/Softmax
final_conv_name="conv5_block3_3_conv/BiasAdd:0"  

model_path = 'models/COVIDNet-CXR-Large/model' #models/COVIDNet-CXR-2/model
model_path2 = 'models/COVIDNet-CXR-Large/model-8485'

metaname = 'model.meta'
ckptname = 'model'
args_n_classes = 2

args_testfolder = 'data/test'
args_trainfile = 'labels/train_COVIDx9B.txt'
args_testfile = 'labels/test_COVIDx9B.txt'

args_out_tensorname = 'dense_3/Softmax:0' #norm_dense_2/Softmax:0
args_logit_tensorname = 'norm_dense_2/MatMul:0'
args_is_severity_model = False
args_is_medusa_backbone = False

args_in_tensorname = 'input_1:0'
args_in_tensorname_medusa = 'input_1:0'
args_input_size = 224 # 480
args_input_size_medusa = 256
args_top_percent = 0.08

# save image
args_impath = "assets/ex-covid.jpeg"
args_pred_class = 'COVID-19'
args_outdir = './'

mapping = {'normal': 0, 'pneumonia': 1, 'COVID-19': 2}
inv_mapping = {0: 'normal', 1: 'pneumonia', 2: 'COVID-19'}



# if __name__ == '__main__':
sess = tf.Session()

saver = tf.train.import_meta_graph(model_path + '.meta')

saver.restore(sess, model_path2)

graph = tf.get_default_graph()

image_tensor = graph.get_tensor_by_name(args_in_tensorname)
gradCam = GradCAM(graph=graph, classes = [0,1,2], outLayer=args_out_tensorname, targetLayer=final_conv_name)

grads = gradCam.compute_grads()

x, origin_im = process_image_file(args_impath, args_top_percent, args_input_size)
img_arr = np.asanyarray(x)
size_upsample = (origin_im.shape[1],origin_im.shape[0]) # (w, h)

x = x.astype('float32') / 255.0
output, grads_val = sess.run([gradCam.target, grads[mapping[args_pred_class]]], feed_dict={image_tensor: np.expand_dims(x, axis=0)})

cam3 = generate_cam(output[0],grads_val[0],size_upsample)


# Overlay cam on image
cam3 = np.uint8(255*cam3)
cam3 = cv2.applyColorMap(cam3, cv2.COLORMAP_JET)

new_im = cam3*0.3 + origin_im*0.5

im_name = args_impath.split("/")[-1]
ext = im_name.split(".")[-1]

# Save the GradCAM
cv2.imwrite(os.path.join(args_outdir, "{}_{}png".format(args_pred_class, im_name.rstrip(ext))), new_im)

print("GradCAM image is save in ", args_outdir)


Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from models/COVIDNet-CXR-Large/model-8485
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
GradCAM image is save in  ./


## Test once

In [14]:
# # Test for example image


# '''/////////////////////////main_grad use/////////////////////////////'''
# temp_name = './assets/ex-covid.jpeg'
# #     radiopaedia_org_covid-19-pneumonia-7_85703_0-dcm.nii.gz
# print("Current img:", temp_name)
# img_m, new_im_2, cam3_resize = main_grad(temp_name)
# '''/////////////////////////main_grad use/////////////////////////////'''


# '''/////////////////////////show grad-cam area/////////////////////////////'''
# img = cam3_resize
# img2= img_m
# gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# gray2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)

# # ret, binary = cv2.threshold(gray,127,255,cv2.THRESH_BINARY)
# # ret, binary = cv2.threshold(gray,120,255,cv2.THRESH_BINARY)
# binary = cv2.inRange(gray,50,80)
# # binary_lung = cv2.inRange(gray2,15,70)


# # print("ret: ", ret)
# cv2_imshow(binary)

# # cv2_imshow(gray2)
# # cv2_imshow(binary_lung)

# print("cam_rec")
# contours, hierarchy = cv2.findContours(binary,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
# if len(contours) > 0  :
#   cnt = contours[0]
#   x, y, w, h = cv2.boundingRect(cnt)
# else:
#   x, y, w, h = 0, 0, 0, 0
# img_new = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 255), 1)
# cv2_imshow(img_new)

# # print("lung_rec")
# # contours2, hierarchy2 = cv2.findContours(binary_lung,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
# # cnt2 = contours2[0]
# # x2, y2, w2, h2 = cv2.boundingRect(cnt2)
# # img_new2 = cv2.rectangle(img2, (x2, y2), (x2 + w2, y2 + h2), (0, 255, 255), 1)
# # cv2_imshow(img_new2)
# '''/////////////////////////show grad-cam area/////////////////////////////'''

# '''/////////////////////////main-result/////////////////////////////'''

# image = new_im_2
# # draw the ground-truth bounding box along with the predicted

# # bounding box

# detection1 = [50, 50, 200, 150] #15, 15, 200, 200
# # detection1 = [x2, y2, x2 + w2, y2 + h2]
# detection2 = [x, y, x + w, y + h]
# start_point1 = [50, 50]  #10,10
# end_point1 = [200, 150]  #200,150
# # start_point1 = [x2, y2]   # 137
# # end_point1 = [x2+w2, y2+h2]  # 188


# color1 = (0, 255, 0)
# start_point2 = [x, y]   # 137
# end_point2 = [x+w, y+h]  # 188
# color2 = (0, 0, 255)
# thickness = 2

# cv2.rectangle(image, start_point1, end_point1, color1, thickness)
# cv2.rectangle(image, start_point2, end_point2, color2, thickness)



# # compute the intersection over union and display it
# iou = bb_intersection_over_union(detection1, detection2)
# # iou = bb_intersection_over_union(detection.gt, detection.pred)
# cv2.putText(image, "IoU: {:.4f}".format(iou), (10, 30),
#   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

# # show the output image
# cv2_imshow(image)


# '''/////////////////////////main-result/////////////////////////////'''


## loop for all (10/30) images

In [15]:
# # create folder to save
# !mkdir ./output_10/

In [16]:
# # change into loop

# # for i in range(10):
# #   num = img_num_set[i]
# #   print("Num#:", num)
# #   print(image_name_all[num])
# #   temp_name = pd_reader['filename'][num]
# #   print("Current img:", temp_name)
# #   main_grad('./covid-chestxray-dataset-master/images/' + temp_name)
# #   # main_grad('./covid-chestxray-dataset-master/images/' + image_name_all[num])
# #   print(i, PCR_result_all[num])

# iou_list = []
# for i in range(len(img_num_set)):
#   '''/////////////////////////main_grad use/////////////////////////////'''
#   num = img_num_set[i]
#   temp_name = pd_reader['filename'][num]
#   # temp_name = './assets/ex-covid.jpeg'
#   print("Current img:", temp_name)
#   # img_m = cv.imread(temp_name)
#   # img_m = cv.resize(img_m, (224, 224))
#   # cv2_imshow(img_m)
#   img_m, new_im_2, cam3_resize = main_grad('./covid-chestxray-dataset-master/images/' + temp_name)
#   cv2.imwrite("./output_10/no_"+temp_name, new_im_2)
#   # print(i, PCR_result_all[num])
#   '''/////////////////////////main_grad use/////////////////////////////'''


#   '''/////////////////////////show grad-cam area/////////////////////////////'''


#   img = cam3_resize

#   gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

#   binary = cv2.inRange(gray,45,80)
# #   cv2_imshow(binary)
#   contours, hierarchy = cv2.findContours(binary,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
#   # print(contours)
#   if len(contours) > 0:
#     cnt = contours[0]
#     x, y, w, h = cv2.boundingRect(cnt)
#     img_new = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 255), 1)
# #     cv2_imshow(img_new)
#   else:
#     x, y, w, h = 0, 0, 0, 0
#   '''/////////////////////////show grad-cam area/////////////////////////////'''

#   '''/////////////////////////main-result/////////////////////////////'''

#   image = new_im_2
#   # draw the ground-truth bounding box along with the predicted

#   # bounding box

#   # # left lung
#   # rect = plt.Rectangle((0.1,10), 20, 30, fill=False, edgecolor = 'red',linewidth=1)  
#   # # right lung
#   # rect2 = plt.Rectangle((25,10), 20, 30, fill=False, edgecolor = 'red',linewidth=1)  
#   # rect = plt.Rectangle((0.1,10), 45, 30, fill=False, edgecolor = 'red',linewidth=1)
#   detection1 = [50, 50, 200, 150] #15, 15, 200, 200
#   detection2 = [x, y, x + w, y + h]
#   start_point1 = [50, 50]  #10,10
#   end_point1 = [200, 150]  #200,150
#   color1 = (0, 255, 0)
#   start_point2 = [x, y]   # 137
#   end_point2 = [x+w, y+h]  # 188
#   color2 = (0, 0, 255)
#   thickness = 2

#   cv2.rectangle(image, start_point1, end_point1, color1, thickness)
#   cv2.rectangle(image, start_point2, end_point2, color2, thickness)



#   # compute the intersection over union and display it
#   iou = bb_intersection_over_union(detection1, detection2)
#   # iou = bb_intersection_over_union(detection.gt, detection.pred)
#   cv2.putText(image, "IoU: {:.4f}".format(iou), (10, 30),
#     cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

#   # show the output image
# #   cv2_imshow(image)
#   iou_list.append(iou)
#   cv2.imwrite("./output_10/IOU_"+temp_name, image)


#   '''/////////////////////////main-result/////////////////////////////'''


In [17]:
# print(iou_list)
# dict = {}
# i = 0
# for num in img_num_set:
#   dict[num] = iou_list[i]
#   i += 1
# dict

# df = pd.DataFrame.from_dict(dict,orient='index',columns=['IOU'])
# df = df.reset_index().rename(columns = {'index':'num'})


# # save into excel
# df.to_excel("./output_10/10_org_iou.xlsx", index=False)

In [18]:
# save
# !cp -r ./output/ 'drive/MyDrive/covid/IOU_ex/'

## Loop All 900

In [43]:
# create folder to save
!mkdir ./output_900_iou_bigger/

In [48]:
iou_list_all = []
cal_red_list = []
for i in range(len(pd_reader)):
  '''/////////////////////////main_grad use/////////////////////////////'''
  num = i
  temp_name = pd_reader['filename'][num]
  # temp_name = './assets/ex-covid.jpeg'
  print("Current img:", temp_name)
  temp_img = cv.imread('./covid-chestxray-dataset-master/images/' + temp_name)
  if temp_img is None:
    temp_name = "white.png"
    img_m, new_im_2, cam3_resize = main_grad(temp_name)

  # img_m = cv.resize(img_m, (224, 224))
  # cv2_imshow(img_m)
  else:
    img_m, new_im_2, cam3_resize = main_grad('./covid-chestxray-dataset-master/images/' + temp_name)
    
#   write imgs
#   cv2.imwrite("./output_900_iou_smaller/"+temp_name,img_m)
  cv2.imwrite("./output_900_iou_bigger/ex-"+temp_name,new_im_2)
  # print(i, PCR_result_all[num])
  '''/////////////////////////main_grad use/////////////////////////////'''


#   '''/////////////////////////show grad-cam area/////////////////////////////'''
#   img = cam3_resize

#   gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#   hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

#   binary = cv2.inRange(gray,45,80)
# #   upper_blue = np.array([110,255,255])

# #   binary = cv2.inRange(hsv, np.array([0,0,0]), upper_blue)
    
#   cal_red = 0
#   total_cal = 0
#   for j in range(binary.shape[0]):
#     for i in range(binary.shape[1]):
#       if binary[j][i] == 255:
#         cal_red += 1
#       total_cal += 1
#   per_red = cal_red / total_cal
#   cal_red_list.append(per_red)
# #   print("total_cal, ", total_cal)
# #   print("cal_red", per_red)
# #   print("blud per: ", 1-per_red)

#   # cv2_imshow(binary)
#   contours, hierarchy = cv2.findContours(binary,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
#   # print(contours)
#   if len(contours) > 0:
#     cnt = contours[0]
#     x, y, w, h = cv2.boundingRect(cnt)
#     img_new = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 255), 1)
#     # cv2_imshow(img_new)
#   else:
#     x, y, w, h = 0, 0, 0, 0
#   '''/////////////////////////show grad-cam area/////////////////////////////'''

#   '''/////////////////////////main-result/////////////////////////////'''

#   image = new_im_2
#   # draw the ground-truth bounding box along with the predicted

#   # bounding box
#   detection1 = [50, 50, 200, 150] #15, 15, 200, 200
#   detection2 = [x, y, x + w, y + h]
#   start_point1 = [50, 50]  #10,10
#   end_point1 = [200, 150]  #200,150
#   color1 = (0, 255, 0)
#   start_point2 = [x, y]   # 137
#   end_point2 = [x+w, y+h]  # 188
#   color2 = (0, 0, 255)
#   thickness = 2

#   cv2.rectangle(image, start_point1, end_point1, color1, thickness)
#   cv2.rectangle(image, start_point2, end_point2, color2, thickness)



#   # compute the intersection over union and display it
#   iou = bb_intersection_over_union(detection1, detection2)
#   # iou = bb_intersection_over_union(detection.gt, detection.pred)
#   cv2.putText(image, "IoU: {:.4f}".format(iou), (10, 30),
#     cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

#   # show the output image
#   # cv2_imshow(image)
#   iou_list_all.append(iou)

  
#   if temp_img is None:
#     pass
#   else:
#     cv2.imwrite("./output_900_iou_smaller/IOU_"+temp_name, image)


  '''/////////////////////////main-result/////////////////////////////'''


Current img: auntminnie-a-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg
pred [[4.8324199e-07 6.9753582e-06 9.9999249e-01]]
Prediction: COVID-19
Confidence
Normal: 0.000 Pneumonia: 0.000 Covid-19: 1.000
Current img: auntminnie-b-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg
pred [[6.5234033e-08 2.0346113e-06 9.9999785e-01]]
Prediction: COVID-19
Confidence
Normal: 0.000 Pneumonia: 0.000 Covid-19: 1.000
Current img: auntminnie-c-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg
pred [[5.8104629e-08 2.0348577e-06 9.9999785e-01]]
Prediction: COVID-19
Confidence
Normal: 0.000 Pneumonia: 0.000 Covid-19: 1.000
Current img: auntminnie-d-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg
pred [[2.1693018e-07 1.0585318e-05 9.9998915e-01]]
Prediction: COVID-19
Confidence
Normal: 0.000 Pneumonia: 0.000 Covid-19: 1.000
Current img: nejmc2001573_f1a.jpeg
pred [[9.4741303e-04 7.6580606e-03 9.9139446e-01]]
Prediction: COVID-19
Confidence
Normal: 0.001 Pneumonia: 0

/opt/apps/apps/binapps/anaconda3/2019.07-numpy-fix/lib/python3.7/site-packages/ipykernel_launcher.py:196: RuntimeWarning: invalid value encountered in true_divide


Current img: SARS-10.1148rg.242035193-g04mr34g0-Fig8c-day10.jpeg
pred [[8.1919234e-06 9.9998522e-01 6.5975983e-06]]
Prediction: pneumonia
Confidence
Normal: 0.000 Pneumonia: 1.000 Covid-19: 0.000


/opt/apps/apps/binapps/anaconda3/2019.07-numpy-fix/lib/python3.7/site-packages/ipykernel_launcher.py:196: RuntimeWarning: invalid value encountered in true_divide


Current img: SARS-10.1148rg.242035193-g04mr34g04a-Fig4a-day7.jpeg
pred [[0.5552118  0.08515265 0.35963556]]
Prediction: normal
Confidence
Normal: 0.555 Pneumonia: 0.085 Covid-19: 0.360


...

pred [[0.00839365 0.09434158 0.8972648 ]]
Prediction: COVID-19
Confidence
Normal: 0.008 Pneumonia: 0.094 Covid-19: 0.897
Current img: 16883_1_1.png
pred [[8.891618e-04 6.625607e-02 9.328548e-01]]
Prediction: COVID-19
Confidence
Normal: 0.001 Pneumonia: 0.066 Covid-19: 0.933
Current img: 2edb88df42cab5e5fbc18b3965e0bd_jumbo.jpeg
pred [[1.4569833e-04 2.1920669e-01 7.8064758e-01]]
Prediction: COVID-19
Confidence
Normal: 0.000 Pneumonia: 0.219 Covid-19: 0.781
Current img: 16858_1_1.png
pred [[2.9720506e-05 1.9411837e-01 8.0585188e-01]]
Prediction: COVID-19
Confidence
Normal: 0.000 Pneumonia: 0.194 Covid-19: 0.806
Current img: 16858_3_1.png
pred [[5.329590e-05 9.699784e-03 9.902469e-01]]
Prediction: COVID-19
Confidence
Normal: 0.000 Pneumonia: 0.010 Covid-19: 0.990
Current img: 16865_1_1.jpg
pred [[0.0000000e+00 3.5922556e-33 1.0000000e+00]]
Prediction: COVID-19
Confidence
Normal: 0.000 Pneumonia: 0.000 Covid-19: 1.000
Current img: 16865_1_2.jpg
pred [[9.8419778e-20 1.9831923e-11 1.0000000e

/opt/apps/apps/binapps/anaconda3/2019.07-numpy-fix/lib/python3.7/site-packages/ipykernel_launcher.py:196: RuntimeWarning: invalid value encountered in true_divide


pred [[0.00270937 0.07752813 0.91976243]]
Prediction: COVID-19
Confidence
Normal: 0.003 Pneumonia: 0.078 Covid-19: 0.920
Current img: 000001-272.jpg
pred [[0.01080808 0.9832003  0.00599154]]
Prediction: pneumonia
Confidence
Normal: 0.011 Pneumonia: 0.983 Covid-19: 0.006
Current img: 000002-268.jpg


/opt/apps/apps/binapps/anaconda3/2019.07-numpy-fix/lib/python3.7/site-packages/ipykernel_launcher.py:196: RuntimeWarning: invalid value encountered in true_divide


pred [[0.0553717  0.27335957 0.67126876]]
Prediction: COVID-19
Confidence
Normal: 0.055 Pneumonia: 0.273 Covid-19: 0.671


In [42]:
# total_cal

50176

In [45]:
print(iou_list_all)
dict = {}

for i in range(len(iou_list_all)):
  dict[i] = iou_list_all[i]

# dict

[0.0, 0.0, 0.0, 0.0, 0.3772597021261542, 0.0, 0.0, 0.0, 0.223553385096441, 0.05232885276648953, 0.0, 0.0, 0.0, 0.1912355374378035, 0.0, 0.0, 0.0, 0.08250846158758542, 0.010818962690971084, 0.03840909090909091, 0.40947778225265136, 0.0, 0.0, 0.07303811997825167, 0.03232821959677894, 0.10980140347658579, 0.06783422588893713, 0.2414445399828031, 0.5198458809858725, 0.0, 0.4077718899823366, 0.008823697931587693, 0.014818067065473903, 0.0, 0.03624180076447549, 0.0, 0.0, 0.0, 0.11398268926002682, 0.0, 0.0, 0.0, 0.0, 0.3822016703628557, 0.11398268926002682, 0.0, 0.13516539440203562, 0.0, 0.03488295849452495, 0.0, 0.11464237946627603, 0.04552499031383185, 0.0798636155006229, 0.0, 0.0, 0.0, 0.0, 0.036485253876558224, 0.05881581535637007, 0.0, 0.0, 0.3524995920143611, 0.033047013310602585, 0.37192772116125716, 0.26343931466681997, 0.0, 0.0, 0.07988655164263768, 0.08488661996913213, 0.0, 0.0, 0.010435937574118875, 0.0, 0.0, 0.0, 0.0, 0.3362220400515283, 0.03840909090909091, 0.009442003803029309, 

In [46]:
print(cal_red_list)
dict2 = {}

for i in range(len(cal_red_list)):
  dict2[i] = cal_red_list[i]

# dict

[0.10532924107142858, 0.08410395408163265, 0.04761240433673469, 0.08396444515306123, 0.21051897321428573, 0.06883769132653061, 0.05540497448979592, 0.1359016262755102, 0.24944196428571427, 0.056939572704081634, 0.0, 0.0, 0.0, 0.013193558673469387, 0.0, 0.0, 0.0, 0.061025191326530615, 0.0419921875, 0.009028220663265306, 0.10799984056122448, 0.02060746173469388, 0.036232461734693876, 0.04683514030612245, 0.030652104591836735, 0.1619499362244898, 0.08577806122448979, 0.13526387117346939, 0.08759167729591837, 0.02459343112244898, 0.03555484693877551, 0.06196189413265306, 0.10331632653061225, 0.37625558035714285, 0.1250597895408163, 0.08765146683673469, 0.06949537627551021, 0.0, 0.008988360969387755, 0.0, 0.07025271045918367, 0.06361607142857142, 0.044224330357142856, 0.03099091198979592, 0.008928571428571428, 0.0, 0.01833545918367347, 0.0, 0.03497688137755102, 0.0, 0.05805564413265306, 0.06955516581632654, 0.0690170599489796, 0.0, 0.13008211096938777, 0.05121970663265306, 0.0, 0.0821308992

In [47]:
# df = pd.DataFrame(dict, columns=['num', 'IOU'])

df = pd.DataFrame.from_dict(dict,orient='index',columns=['IOU'])
df = df.reset_index().rename(columns = {'index':'num'})

df2 = pd.DataFrame.from_dict(dict2,orient='index',columns=['red_cal'])
df2 = df2.reset_index().rename(columns = {'index':'num'})


# save into excel
df.to_excel("900_org_iou_smaller.xlsx", index=False)
# save into excel
df2.to_excel("900_org_red_smaller.xlsx", index=False)

In [24]:
# import os, tarfile
 
# import os
# from google.colab import files

# def make_targz_one_by_one(output_filename, source_dir):
#   tar = tarfile.open(output_filename,"w")
#   for root,dir_name,files_list in os.walk(source_dir):
#     for file in files_list:
#       pathfile = os.path.join(root, file)
#       tar.add(pathfile)
#   tar.close()
 
#   files.download(output_filename)
 
 
# make_targz_one_by_one('output_all_iou_org', '/content/output_all_iou_org/')

In [25]:
# save
# !cp -r ./output_900_iou_org/ 'drive/MyDrive/covid/IOU_ex/'

# Robust

## Test once

In [26]:
# # Test for example image

# '''/////////////////////////main_grad use/////////////////////////////'''
# temp_name = 'c94ef38ddb2d94aa11828716777139_jumbo.jpeg'
# # ./assets/ex-covid.jpeg
# print("Current img:", temp_name)
# img_m, new_im_2, cam3_resize = main_grad('./covid-chestxray-dataset-master/images/'+temp_name)
# '''/////////////////////////main_grad use/////////////////////////////'''


# '''/////////////////////////show grad-cam area/////////////////////////////'''
# img = cam3_resize
# img2= img_m
# gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# gray2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)

# binary = cv2.inRange(gray,50,80)

# cv2_imshow(binary)


# print("cam_rec")
# contours, hierarchy = cv2.findContours(binary,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

# cnt = contours[0]
# x, y, w, h = cv2.boundingRect(cnt)
# img_new = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 255), 1)
# cv2_imshow(img_new)

# '''/////////////////////////show grad-cam area/////////////////////////////'''


# print("x, y", x, y)
# print("x+w, y+h", x+w, y+h)

# for i in range(img_m.shape[0]):
#   for j in range(img_m.shape[1]):
#     if (j>=x and j<=x+w) and (i>=y and i<=y+h):
#       img_m[i][j] = [255, 255, 255]
# cv2_imshow(img_m)

In [27]:
# # write the image to the current folder temperarily
# cv2.imwrite("test1.jpg", img_m)

True

In [28]:
# # test1 test
# new_name = 'test1.jpg'
# # print("Current img:", temp_name)

# img_m_new, new_im_2_new, cam3_resize_new = main_grad(new_name)

pred [[0.15720429 0.51102364 0.33177206]]
Prediction: pneumonia
Confidence
Normal: 0.157 Pneumonia: 0.511 Covid-19: 0.332


/opt/apps/apps/binapps/anaconda3/2019.07-numpy-fix/lib/python3.7/site-packages/ipykernel_launcher.py:196: RuntimeWarning: invalid value encountered in true_divide


Result above shows that when the area removed, the prediction not change in this case, but only lower the probability of the covid19, while the area of the most likely covid part **changed**.

## Loop for 30

In [29]:
# # !mkdir ./output_org/
# !mkdir ./output_robust/

In [30]:
# # change into loop

# iou_list = []
# for i in range(len(img_num_set)):
#   '''/////////////////////////main_grad use/////////////////////////////'''
#   num = img_num_set[i]
#   temp_name = pd_reader['filename'][num]
#   # temp_name = './assets/ex-covid.jpeg'
#   print("Current img:", temp_name)
#   # img_m = cv.imread(temp_name)
#   # img_m = cv.resize(img_m, (224, 224))
#   # cv2_imshow(img_m)
#   img_m, new_im_2, cam3_resize = main_grad('./covid-chestxray-dataset-master/images/' + temp_name)
#   # print(i, PCR_result_all[num])
#   '''/////////////////////////main_grad use/////////////////////////////'''


#   '''/////////////////////////show grad-cam area/////////////////////////////'''


#   img = cam3_resize

#   gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

#   binary = cv2.inRange(gray,45,80)
#   cv2_imshow(binary)
#   contours, hierarchy = cv2.findContours(binary,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
#   # print(contours)
#   if len(contours) > 0:
#     cnt = contours[0]
#     x, y, w, h = cv2.boundingRect(cnt)
#     img_new = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 255), 1)
#     cv2_imshow(img_new)
#   else:
#     x, y, w, h = 0, 0, 0, 0


  
#   '''/////////////////////////show grad-cam area/////////////////////////////'''

#   '''/////////////////////////Robust/////////////////////////////'''
#   for i in range(img_m.shape[0]):
#     for j in range(img_m.shape[1]):
#       if (j>=x and j<=x+w) and (i>=y and i<=y+h):
#         img_m[i][j] = [255, 255, 255]
#   cv2_imshow(img_m)

#   # write the image to the current folder temperarily
#   cv2.imwrite("./output_robust/rob-"+temp_name, img_m)

#   new_name = "./output_robust/rob-"+temp_name
#   # print("Current img:", temp_name)

#   img_m_new, new_im_2_new, cam3_resize_new = main_grad(new_name)

#   '''/////////////////////////Robust/////////////////////////////'''

#   '''/////////////////////////main-result/////////////////////////////'''

#   image = new_im_2_new
#   # draw the ground-truth bounding box along with the predicted

#   img = cam3_resize_new
#   gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

#   binary = cv2.inRange(gray,45,80)
#   cv2_imshow(binary)
#   contours, hierarchy = cv2.findContours(binary,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
#   # print(contours)
#   if len(contours) > 0:
#     cnt = contours[0]
#     x, y, w, h = cv2.boundingRect(cnt)
#     img_new = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 255), 1)
#     cv2_imshow(img_new)
#   else:
#     x, y, w, h = 0, 0, 0, 0

#   # bounding box

#   detection1 = [50, 50, 200, 150] #15, 15, 200, 200
#   detection2 = [x, y, x + w, y + h]
#   start_point1 = [50, 50]  #10,10
#   end_point1 = [200, 150]  #200,150
#   color1 = (0, 255, 0)
#   start_point2 = [x, y]   # 137
#   end_point2 = [x+w, y+h]  # 188
#   color2 = (0, 0, 255)
#   thickness = 2

#   cv2.rectangle(image, start_point1, end_point1, color1, thickness)
#   cv2.rectangle(image, start_point2, end_point2, color2, thickness)



#   # compute the intersection over union and display it
#   iou = bb_intersection_over_union(detection1, detection2)
#   # iou = bb_intersection_over_union(detection.gt, detection.pred)
#   cv2.putText(image, "IoU: {:.4f}".format(iou), (10, 30),
#     cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

#   # show the output image
#   cv2_imshow(image)
#   iou_list.append(iou)
#   cv2.imwrite("./output_robust/IOU_"+temp_name, image)


#   '''/////////////////////////main-result/////////////////////////////'''


In [31]:
# print(iou_list)

In [32]:
# dict = {}
# i = 0
# for num in img_num_set:
#   dict[num] = iou_list[i]
#   i += 1
# dict

In [33]:
# # df = pd.DataFrame(dict, columns=['num', 'IOU'])

# df = pd.DataFrame.from_dict(dict,orient='index',columns=['IOU'])
# df = df.reset_index().rename(columns = {'index':'num'})


# # save into excel
# df.to_excel("30_org_iou.xlsx", index=False)

In [34]:
# # save
# !cp -r ./output_robust/ 'drive/MyDrive/covid/IOU_ex/'

## ROBUST Loop All -900

Use smaller here

In [9]:
# create folder to save
!mkdir ./output_900_iou_robust/

In [ ]:
iou_list_all2 = []

pred_list_all = []
pred_list_all2 = []
cls_list_all = []
cls1_list_all = []
cls2_list_all = []
change0_list_all = []
change_list_all = []
for i in range(len(pd_reader)):
  '''/////////////////////////main_grad use/////////////////////////////'''
  num = i
  temp_name = pd_reader['filename'][num]
  print("Current img:", temp_name)
  temp_img = cv.imread('./covid-chestxray-dataset-master/images/' + temp_name)
  if temp_img is None:
    temp_name = "white.png"
    img_m, new_im_2, cam3_resize, pred0 = main_grad(temp_name)
  else:
    img_m, new_im_2, cam3_resize, pred0 = main_grad('./covid-chestxray-dataset-master/images/' + temp_name)
  pred_list_all.append(pred0)
  pred_max0 = pred0[0].max()
  if pred0[0][0] == pred_max0:
    # nomal
    cls0 = 0
  elif pred0[0][1] == pred_max0:
    # non-covid
    cls0 = 1
  else:
    # covid
    cls0 = 2 
  cls_list_all.append(cls0)
  '''/////////////////////////main_grad use/////////////////////////////'''


  '''/////////////////////////show grad-cam area/////////////////////////////'''


  img = cam3_resize

  gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

  binary = cv2.inRange(gray,45,80)
  # cv2_imshow(binary)
  contours, hierarchy = cv2.findContours(binary,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
  # print(contours)
  if len(contours) > 0:
    cnt = contours[0]
    x, y, w, h = cv2.boundingRect(cnt)
    img_new = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 255), 1)
    # cv2_imshow(img_new)
  else:
    x, y, w, h = 0, 0, 0, 0


  
  '''/////////////////////////show grad-cam area/////////////////////////////'''

  '''/////////////////////////Robust/////////////////////////////'''
  img1 = img_m
  img2 = img_m
  for i in range(img1.shape[0]):
    for j in range(img1.shape[1]):
      if (j>=x and j<=x+w) and (i>=y and i<=y+h):
        img1[i][j] = [255, 255, 255]
  # cv2_imshow(img_m)

  # write the image to the current folder temperarily
  cv2.imwrite("./output_900_iou_robust/rob1-"+temp_name, img1)

  new_name = "./output_900_iou_robust/rob1-"+temp_name
  # print("Current img:", temp_name)

  img_m_new, new_im_2_new, cam3_resize_new, pred = main_grad(new_name)
  pred_list_all2.append(pred)
  pred_max = pred[0].max()
  if pred[0][0] == pred_max:
    # nomal
    cls1 = 0
  elif pred[0][1] == pred_max:
    # non-covid
    cls1 = 1
  else:
    # covid
    cls1 = 2
  cls1_list_all.append(cls1)

  #  robust half - 1/4
  for i in range(img2.shape[0]):
    for j in range(img2.shape[1]):
      if (j>=x and j<=x+w/2) and (i>=y and i<=y+h/2):
        img2[i][j] = [255, 255, 255]
  cv2.imwrite("./output_900_iou_robust/rob2-"+temp_name, img2)

  new_name2 = "./output_900_iou_robust/rob2-"+temp_name
  # print("Current img:", temp_name)

  img_m_new2, new_im_2_new2, cam3_resize_new2, pred2 = main_grad(new_name2)
  pred_list_all2.append(pred2)
  pred_max = pred2[0].max()
  if pred2[0][0] == pred_max:
    # nomal
    cls2 = 0
  elif pred2[0][1] == pred_max:
    # non-covid
    cls2 = 1
  else:
    # covid
    cls2 = 2
  cls2_list_all.append(cls2)

  if cls1 == cls0:
    change = 1
  else:
    change = 0

  if cls0 == cls2:
    change2 = 1
  else:
    change2 = 0
  change_list_all.append(change2)
  change0_list_all.append(change)

  '''/////////////////////////Robust/////////////////////////////'''

  '''/////////////////////////main-result/////////////////////////////'''

  image = new_im_2_new2
  # draw the ground-truth bounding box along with the predicted

  img = cam3_resize_new
  gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

  binary = cv2.inRange(gray,45,80)
  # cv2_imshow(binary)
  contours, hierarchy = cv2.findContours(binary,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
  # print(contours)
  if len(contours) > 0:
    cnt = contours[0]
    x, y, w, h = cv2.boundingRect(cnt)
    img_new = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 255), 1)
    # cv2_imshow(img_new)
  else:
    x, y, w, h = 0, 0, 0, 0

  # bounding box

  detection1 = [50, 50, 200, 150] #15, 15, 200, 200
  detection2 = [x, y, x + w, y + h]
  start_point1 = [50, 50]  #10,10
  end_point1 = [200, 150]  #200,150
  color1 = (0, 255, 0)
  start_point2 = [x, y]   # 137
  end_point2 = [x+w, y+h]  # 188
  color2 = (0, 0, 255)
  thickness = 2

  cv2.rectangle(image, start_point1, end_point1, color1, thickness)
  cv2.rectangle(image, start_point2, end_point2, color2, thickness)



  # compute the intersection over union and display it
  iou = bb_intersection_over_union(detection1, detection2)
  # iou = bb_intersection_over_union(detection.gt, detection.pred)
  cv2.putText(image, "IoU: {:.4f}".format(iou), (10, 30),
    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

  # show the output image
  # cv2_imshow(image)
  iou_list_all2.append(iou)
  cv2.imwrite("./output_900_iou_robust/IOU_half_"+temp_name, image)


  '''/////////////////////////main-result/////////////////////////////'''


Current img: auntminnie-a-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg
pred [[4.8324199e-07 6.9753582e-06 9.9999249e-01]]
Prediction: COVID-19
Confidence
Normal: 0.000 Pneumonia: 0.000 Covid-19: 1.000
pred [[1.1872621e-07 2.0007803e-06 9.9999785e-01]]
Prediction: COVID-19
Confidence
Normal: 0.000 Pneumonia: 0.000 Covid-19: 1.000
pred [[1.1872621e-07 2.0007803e-06 9.9999785e-01]]
Prediction: COVID-19
Confidence
Normal: 0.000 Pneumonia: 0.000 Covid-19: 1.000
Current img: auntminnie-b-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg
pred [[6.5234033e-08 2.0346113e-06 9.9999785e-01]]
Prediction: COVID-19
Confidence
Normal: 0.000 Pneumonia: 0.000 Covid-19: 1.000
pred [[1.0937270e-07 2.8183283e-06 9.9999702e-01]]
Prediction: COVID-19
Confidence
Normal: 0.000 Pneumonia: 0.000 Covid-19: 1.000
pred [[1.0937270e-07 2.8183283e-06 9.9999702e-01]]
Prediction: COVID-19
Confidence
Normal: 0.000 Pneumonia: 0.000 Covid-19: 1.000
Current img: auntminnie-c-2020_01_28_23_51_6665_2020

/opt/apps/apps/binapps/anaconda3/2019.07-numpy-fix/lib/python3.7/site-packages/ipykernel_launcher.py:196: RuntimeWarning: invalid value encountered in true_divide


pred [[0.12868325 0.7423369  0.12897971]]
Prediction: pneumonia
Confidence
Normal: 0.129 Pneumonia: 0.742 Covid-19: 0.129
Current img: nejmc2001573_f1b.jpeg


/opt/apps/apps/binapps/anaconda3/2019.07-numpy-fix/lib/python3.7/site-packages/ipykernel_launcher.py:196: RuntimeWarning: invalid value encountered in true_divide


pred [[1.58395744e-06 1.46173725e-05 9.99983788e-01]]
Prediction: COVID-19
Confidence
Normal: 0.000 Pneumonia: 0.000 Covid-19: 1.000
pred [[3.4862812e-06 1.4439475e-05 9.9998212e-01]]
Prediction: COVID-19
Confidence
Normal: 0.000 Pneumonia: 0.000 Covid-19: 1.000
pred [[3.4862812e-06 1.4439475e-05 9.9998212e-01]]
Prediction: COVID-19
Confidence
Normal: 0.000 Pneumonia: 0.000 Covid-19: 1.000
Current img: ARDSSevere.png
pred [[7.5563739e-06 2.4952891e-03 9.9749714e-01]]
Prediction: COVID-19
Confidence
Normal: 0.000 Pneumonia: 0.002 Covid-19: 0.997
pred [[1.8815363e-07 9.6420190e-05 9.9990344e-01]]
Prediction: COVID-19
Confidence
Normal: 0.000 Pneumonia: 0.000 Covid-19: 1.000
pred [[1.8815363e-07 9.6420190e-05 9.9990344e-01]]
Prediction: COVID-19
Confidence
Normal: 0.000 Pneumonia: 0.000 Covid-19: 1.000
Current img: lancet-case2a.jpg
pred [[9.1949834e-05 2.5650894e-03 9.9734300e-01]]
Prediction: COVID-19
Confidence
Normal: 0.000 Pneumonia: 0.003 Covid-19: 0.997
pred [[9.6117816e-05 2.58937

...

pred [[8.2219428e-01 1.7780563e-01 7.2851840e-08]]
Prediction: normal
Confidence
Normal: 0.822 Pneumonia: 0.178 Covid-19: 0.000
Current img: DE488FE1-0C44-428B-B67A-09741C1214C0.jpeg


/opt/apps/apps/binapps/anaconda3/2019.07-numpy-fix/lib/python3.7/site-packages/ipykernel_launcher.py:196: RuntimeWarning: invalid value encountered in true_divide


pred [[3.0218247e-02 9.6978170e-01 7.7116791e-10]]
Prediction: pneumonia
Confidence
Normal: 0.030 Pneumonia: 0.970 Covid-19: 0.000
pred [[2.9288638e-01 7.0711339e-01 1.9809055e-07]]
Prediction: pneumonia
Confidence
Normal: 0.293 Pneumonia: 0.707 Covid-19: 0.000
pred [[2.9288638e-01 7.0711339e-01 1.9809055e-07]]
Prediction: pneumonia
Confidence
Normal: 0.293 Pneumonia: 0.707 Covid-19: 0.000
Current img: 7C69C012-7479-493F-8722-ABC29C60A2DD.jpeg
pred [[2.1691234e-03 9.2330633e-04 9.9690753e-01]]
Prediction: COVID-19
Confidence
Normal: 0.002 Pneumonia: 0.001 Covid-19: 0.997
pred [[0.00731862 0.0023611  0.99032027]]
Prediction: COVID-19
Confidence
Normal: 0.007 Pneumonia: 0.002 Covid-19: 0.990
pred [[0.00731862 0.0023611  0.99032027]]
Prediction: COVID-19
Confidence
Normal: 0.007 Pneumonia: 0.002 Covid-19: 0.990
Current img: 44C8E3D6-20DA-42E9-B33B-96FA6D6DE12F.jpeg
pred [[0.010555   0.02829694 0.961148  ]]
Prediction: COVID-19
Confidence
Normal: 0.011 Pneumonia: 0.028 Covid-19: 0.961
pred

/opt/apps/apps/binapps/anaconda3/2019.07-numpy-fix/lib/python3.7/site-packages/ipykernel_launcher.py:196: RuntimeWarning: invalid value encountered in true_divide


pred [[8.7862176e-01 1.2137613e-01 2.0467155e-06]]
Prediction: normal
Confidence
Normal: 0.879 Pneumonia: 0.121 Covid-19: 0.000


In [ ]:
print(pred_list_all)

In [ ]:
print(pred_list_all2)

In [ ]:
print(iou_list_all2)
dict2 = {}

for i in range(len(pd_reader)):
  dict2[i] = iou_list_all2[i]

# dict

In [ ]:
# print(cls1_list_all)
dict2_cls0 = {}
dict2_cls1 = {}
dict2_cls2 = {}
dict_change = {}
dict_change2 = {}

for i in range(len(pd_reader)):
  dict2_cls0[i] = cls_list_all[i]
  dict2_cls1[i] = cls1_list_all[i]
  dict2_cls2[i] = cls2_list_all[i]
  dict_change[i] = change0_list_all[i]
  dict_change2[i] = change_list_all[i]

# dict

In [ ]:
# df = pd.DataFrame(dict, columns=['num', 'IOU'])

df = pd.DataFrame.from_dict(dict2,orient='index',columns=['IOU'])
df = df.reset_index().rename(columns = {'index':'num'})

df2 = pd.DataFrame.from_dict(dict2_cls1,orient='index',columns=['cls1'])
df2 = df2.reset_index().rename(columns = {'index':'num'})

df3 = pd.DataFrame.from_dict(dict2_cls2,orient='index',columns=['cls2'])
df3 = df3.reset_index().rename(columns = {'index':'num'})

df0 = pd.DataFrame.from_dict(dict2_cls0,orient='index',columns=['cls0'])
df0 = df0.reset_index().rename(columns = {'index':'num'})

df4 = pd.DataFrame.from_dict(dict_change,orient='index',columns=['change_cls1_cls0'])
df4 = df4.reset_index().rename(columns = {'index':'num'})

df5 = pd.DataFrame.from_dict(dict_change,orient='index',columns=['change_cls2_cls0'])
df5 = df5.reset_index().rename(columns = {'index':'num'})


# save into excel
df.to_excel("900_robust_half_iou.xlsx", index=False)

df0.to_excel("900_robust_cls0_org.xlsx", index=False)
df2.to_excel("900_robust_cls1.xlsx", index=False)
df3.to_excel("900_robust_cls2.xlsx", index=False)
df4.to_excel("900_robust_change_cls1_0.xlsx", index=False)
df5.to_excel("900_robust_change_cls2_0.xlsx", index=False)

In [ ]:
# save
# !cp -r ./output_900_iou_robust/ 'drive/MyDrive/covid/IOU_ex/'